# 任務: 爬取停車場地段號、坪數、地主、停車價格。

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### 1. 爬取停車場地址及停車費 (以臺北市為例)
##### 車麻吉上之資訊來源為app後端資料庫，並非網路上公開之資料，一般人沒有權限用程式爬取，
##### 故先使用臺北市停車管理工程處網站之臺北市公有路外停車場資料。

In [53]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://pma.gov.taipei/News.aspx?n=6833ECE829BE5990&sms=504AB58CAE8A1C62&page=1&PageSize=200'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

districts_html = soup.find_all('td', {'data-title': '行政區'})
addresses_html = soup.find_all('td', {'data-title': '停車場地址'})
fees_html = soup.find_all('td', {'data-title': '費率'})

districts = [district.text for district in districts_html]
addresses = [address.text for address in addresses_html]
fees = [fee.text for fee in fees_html]

# 轉成dataframe
df = pd.DataFrame({'行政區': districts, '地址': addresses, '費率':fees})
df

,行政區,地址,費率
0,臺北市大安區,信義路3段134巷及建國南路2段25巷交叉口,計時20元
1,臺北市內湖區,行善路266、268號地下,小型車：計時30元，機車：計時10元(當日當次最高收費20元，隔日另計)
2,臺北市信義區,大道路116號地下,小型車：計時30元，機車：計時10元(當日當次最高收費20元，隔日另計)
3,臺北市信義區,福德街86號地下,小型車：計時30元，機車：計時10元(當日當次最高收費20元，隔日另計)
4,臺北市內湖區,金湖路46巷底,計時20元
...,...,...,...
195,臺北市中山區,八德路2段158號地下,計時20元
196,臺北市文山區,辛亥路4段103號地下,(8-18)計時20元，(18-8)計時10元
197,臺北市信義區,松壽路1號地下,小型車(9-18)計時30元 ，(18-9)計時10元，機車計時10元(當日當次最高收費上限...
198,臺北市南港區,成福路1號,星期一至五(8-18)計時20元，(18-8)計時30元，假日計時30元\r\n


### 2. 依地址搜尋地段號

##### 若使用手動搜尋，則使用內政部地籍圖資網路便民服務系統
##### 以第一筆資料 (臺北市大安區信義路3段134巷及建國南路2段25巷交叉口) 為例
##### 其地址以兩路交叉口描述，並不夠精確，可能導致程式自動化時無法順利查詢
##### 
##### 若以車麻吉上之地址: 建國南路二段25巷29號 搜尋
![](https://imgur.com/nd8PZzy.png)
##### 可知標的位於瑞安段一小段 0207地號
##### 
##### 可使用程式取代手動操作，實作如下:

In [ ]:

url = "https://easymap.land.moi.gov.tw/Index"

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# 門牌
door_plate_options = soup.find("select", {"name": "doorPlateType"}).find_all("option")
# 縣市
city_options = soup.find("select", {"name": "select_city"}).find_all("option")
# 鄉鎮市區
road_options = soup.find("select", {"name": "select_road"}).find_all("option")

data = {
    "" = '地政門牌'
    city = '臺北市'
}

response = requests.post(url, data=data)

soup = BeautifulSoup(response.content, 'html.parser')



##### 以第四筆資料 (臺北市信義區福德街86號地下) 為例
![](https://imgur.com/aFZvXPI.png)
##### 問題: 停車場未必有辦建物登記，故無法從門牌查到地段號
##### 
##### 若改以地政雲查詢
![](https://imgur.com/hwSvuZ7.png)
##### 可見標的為319地號
##### 但此為人為辨識，無法由程式自動識別

### 資料處理

In [52]:
# 資料處理
cities = '臺北市'
districts = [district.split('市')[-1] for district in districts]

#addresses = [address.split('號') for address in addresses]
type(df.head()['地址'][0])

str